In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [11]:
analyser = SentimentIntensityAnalyzer()

In [18]:
news_dir = 'data/in/news/'
twitter_dir = 'data/in/twitter/'
influencers = ['BarackObama','cnnbrk', 'KimKardashian']
sources = ['New York Times', 'CNN', 'Washington Post']

In [19]:
def sentiment_analyzer_scores(sentence):
    try:
        score = analyser.polarity_scores(sentence)
        return score['compound']
    except:
        print('Failed to score content:',sentence)
        return 0
    
#     print("{:-<40} {}".format(sentence, str(score)))

In [20]:
#TODO ADD CUSTOM HEURISTICS

In [21]:
def sentimentTwiiter(influencers):
    for influencer in influencers:
        df = pd.read_csv(twitter_dir+influencer+'.csv', parse_dates=['Date'])
        df['sentiment_score'] = df['content'].apply(sentiment_analyzer_scores)
        df.to_csv(twitter_dir+influencer+'.csv', index=False)
    print('done')

In [22]:
def sentimentNews(sources):
    for source in sources:
        df = pd.read_csv(news_dir+source+'.csv', parse_dates=['Date'])
        df['sentiment_score'] = df['title'].apply(sentiment_analyzer_scores)
        df.to_csv(news_dir+source+'.csv', index=False)
    print('done')

In [23]:
#AUTOMATION
sentimentTwiiter(influencers)
sentimentNews(sources)

done
Failed to score content: nan
done


In [33]:
df2 = pd.read_csv('BarackObama.csv', parse_dates=['Date'])


In [34]:
df2

,Date,author,content,number_of_likes,number_of_shares
0,2014-01-08 14:57:00,BarackObama,Make #53 special for #44: http://t.co/leGTnedv...,1512,726
1,2014-01-08 17:58:00,BarackObama,"Job growth has exceeded 200,000 for the sixth ...",816,556
2,2014-01-08 18:46:00,BarackObama,Happening now: President Obama delivers a stat...,491,288
3,2014-01-08 18:54:00,BarackObama,"""That's the longest streak of private sector j...",804,619
4,2014-01-08 18:57:00,BarackObama,"""The fact is we could be much further along .....",1505,1260
...,...,...,...,...,...
2858,2016-12-10 16:35:00,BarackObama,Let's leave our children and grandchildren a p...,12072,3692
2859,2016-12-10 17:57:00,BarackObama,"Last week, more than 190 countries reached a l...",3283,855
2860,2016-12-10 19:43:00,BarackObama,Support the bold steps we're taking against cl...,1979,496
2861,2016-12-10 21:02:00,BarackObama,LIVE: President Obama is speaking at a recepti...,1931,468


In [26]:
df = pd.read_csv('New York Times.csv', parse_dates=['Date'])

In [27]:
df['Score'] = df['title'].apply(sentiment_analyzer_scores)

In [28]:
df

,title,author,Date,Score
0,37 Who Saw Murder Didn't Call the Police - The...,NaN,2014-05-08,-0.6908
1,The Pillow Cure - The New York Times,Penelope Green,2015-03-26,0.0000
2,John Ashbery: By the Book - The New York Times,NaN,2015-05-13,0.0000
3,"Drink to Your Health (in Moderation), the Scie...",Aaron E. Carroll,2015-12-22,0.0000
4,Simple Rules for Healthy Eating - The New York...,Aaron E. Carroll,2015-12-28,0.4019
...,...,...,...,...
30252,How a Trump Adviser Repeatedly Sought a Meetin...,Troy Griggs,NaT,0.0000
30253,"Satellite Images Show 1,800 Buildings Destroye...",Anjali Singhvi,NaT,-0.6808
30254,Trump Could Save More Than $1 Billion Under Hi...,Jesse Drucker,NaT,0.4939
30255,Opinion | The Republican’s Guide to Presidenti...,The Editorial Board,NaT,0.0000


In [30]:
df.to_csv('New York Times.csv', index=False)